In [1]:
%cd ../..

/home/work/AnacondaProjects/combinators


In [2]:
import logging

import numpy as np
import probtorch
import torch

from examples.fep_control import fep_control
from combinators.model import active
from combinators.model import compose, foldable
from combinators.inference import importance, mcmc
from combinators import utils

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [ ]:
target = fep_control.BipedalWalkerAgent(state_dim=12, batch_shape=(10,), trainable=False)
proposal = fep_control.RecognitionAgent(state_dim=12, action_dim=4, observation_dim=24, batch_shape=(10,),
                                        discrete_actions=False, name='BipedalWalkerAgent', trainable=True,
                                        goal=target.goal)
agent = importance.propose(target, proposal)

In [ ]:
episode = active.ActiveEpisode(agent, 'BipedalWalker-v2')

In [ ]:
theta, graph, log_weight = episode(render=True)

In [ ]:
xi, trained_params, elbos = active.active_variational(episode, 100, lr=1e-2, log_estimator=True, patience=100)

07/07/2019 10:30:00 Episode length: 74
07/07/2019 10:30:00 ELBO=-1.86026215e+02 per step at epoch 1
07/07/2019 10:30:01 Episode length: 57
07/07/2019 10:30:01 ELBO=-9.20053940e+01 per step at epoch 2
07/07/2019 10:30:01 Episode length: 36
07/07/2019 10:30:01 ELBO=-7.11589584e+01 per step at epoch 3
07/07/2019 10:30:07 Episode length: 1086
07/07/2019 10:30:11 ELBO=-5.49399300e+01 per step at epoch 4
07/07/2019 10:30:11 Episode length: 65
07/07/2019 10:30:11 ELBO=-7.88883591e+01 per step at epoch 5
07/07/2019 10:30:12 Episode length: 72
07/07/2019 10:30:12 ELBO=-7.78507996e+01 per step at epoch 6
07/07/2019 10:30:12 Episode length: 53
07/07/2019 10:30:12 ELBO=-7.23149490e+01 per step at epoch 7
07/07/2019 10:30:13 Episode length: 64
07/07/2019 10:30:13 ELBO=-6.10128784e+01 per step at epoch 8
07/07/2019 10:30:14 Episode length: 59
07/07/2019 10:30:14 ELBO=-5.81140442e+01 per step at epoch 9
07/07/2019 10:30:24 Episode length: 1601
07/07/2019 10:30:29 ELBO=-5.27227859e+01 per step at epoc

In [ ]:
utils.plot_evidence_bounds(elbos[:, 0].cpu().detach().numpy())

In [ ]:
theta, graph, log_weight = episode(render=True)

In [ ]:
theta

In [ ]:
log_weight

In [ ]:
torch.save(agent, 'examples/fep_control/fep_bipedal_walker_agent.dat')